# Importando bibliotecas

In [2]:
!pip install lxml
!pip install beautifulsoup4
!pip install gensim

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

from gensim.models import Word2Vec
import gensim
from gensim.models import Word2Vec

from collections.abc import Mapping

import bs4 as bs
import urllib.request
import re
import nltk


# Carregando arquivo

In [4]:
file = "/content/B2W-Reviews01.csv"
tabela = pd.read_csv(file, sep=',', encoding='utf-8', low_memory=False)

tabela.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132373 entries, 0 to 132372
Data columns (total 14 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   submission_date        132373 non-null  object 
 1   reviewer_id            132373 non-null  object 
 2   product_id             132373 non-null  object 
 3   product_name           132289 non-null  object 
 4   product_brand          40982 non-null   object 
 5   site_category_lv1      132367 non-null  object 
 6   site_category_lv2      128360 non-null  object 
 7   review_title           132071 non-null  object 
 8   overall_rating         132373 non-null  int64  
 9   recommend_to_a_friend  132355 non-null  object 
 10  review_text            129098 non-null  object 
 11  reviewer_birth_year    126389 non-null  float64
 12  reviewer_gender        128237 non-null  object 
 13  reviewer_state         128382 non-null  object 
dtypes: float64(1), int64(1), object(12)


# Análise Exploratória

In [5]:
# Exibindo DataFrame do Arquivo para análise

tabela

,submission_date,reviewer_id,product_id,product_name,product_brand,site_category_lv1,site_category_lv2,review_title,overall_rating,recommend_to_a_friend,review_text,reviewer_birth_year,reviewer_gender,reviewer_state
0,2018-01-01 00:11:28,d0fb1ca69422530334178f5c8624aa7a99da47907c44de...,132532965,Notebook Asus Vivobook Max X541NA-GO472T Intel...,NaN,Informática,Notebook,Bom,4,Yes,Estou contente com a compra entrega rápida o ú...,1958.0,F,RJ
1,2018-01-01 00:13:48,014d6dc5a10aed1ff1e6f349fb2b059a2d3de511c7538a...,22562178,Copo Acrílico Com Canudo 500ml Rocie,NaN,Utilidades Domésticas,"Copos, Taças e Canecas","Preço imbatível, ótima qualidade",4,Yes,"Por apenas R$1994.20,eu consegui comprar esse ...",1996.0,M,SC
2,2018-01-01 00:26:02,44f2c8edd93471926fff601274b8b2b5c4824e386ae4f2...,113022329,Panela de Pressão Elétrica Philips Walita Dail...,philips walita,Eletroportáteis,Panela Elétrica,ATENDE TODAS AS EXPECTATIVA.,4,Yes,SUPERA EM AGILIDADE E PRATICIDADE OUTRAS PANEL...,1984.0,M,SP
3,2018-01-01 00:35:54,ce741665c1764ab2d77539e18d0e4f66dde6213c9f0863...,113851581,Betoneira Columbus - Roma Brinquedos,roma jensen,Brinquedos,Veículos de Brinquedo,presente mais que desejado,4,Yes,MEU FILHO AMOU! PARECE DE VERDADE COM TANTOS D...,1985.0,F,SP
4,2018-01-01 01:00:28,7d7b6b18dda804a897359276cef0ca252f9932bf4b5c8e...,131788803,"Smart TV LED 43"" LG 43UJ6525 Ultra HD 4K com C...",lg,TV e Home Theater,TV,"Sem duvidas, excelente",5,Yes,"A entrega foi no prazo, as americanas estão de...",1994.0,M,MG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132368,2018-05-31 23:30:50,15f20e95ff44163f3175aaf67a5ae4a94d5030b409e521...,17962233,Carregador De Pilha Sony + 4 Pilhas Aa 2500mah,NaN,Câmeras e Filmadoras,Acessórios para Câmeras e Filmadoras,Ótimo produto!,5,Yes,"Vale muito, estou usando no controle do Xbox e...",1988.0,M,RS
132369,2018-05-31 23:42:25,def7cf9028b0673ab8bca3b1d06e085461fafb88cd48d9...,132631701,Mop Giratório Fit + Refil Extra - At Home,NaN,Utilidades Domésticas,Material de Limpeza,Sensacional,5,Yes,"Prático e barato, super indico o produto para ...",1979.0,F,SP
132370,2018-05-31 23:44:16,7bcbf542f5d7dd9a9a192a6805adba7a7a4c1ce3bf00df...,16095859,Fita Led 5m Rgb 3528 Siliconada Com 300 Leds C...,NaN,Automotivo,Iluminação,Ótimo produto,4,Yes,Chegou antes do prazo previsto e corresponde a...,1979.0,F,PR
132371,2018-05-31 23:46:48,e6fb0b19277d01c2a300c7837a105f3c369377e92f9c19...,6774907,Etiquetas Jurídicas Vade Mecum - Marca Fácil,marca facil,Papelaria,Material de Escritório,O produto não é bom.,1,No,"Material fraco, poderia ser melhor. Ficou deve...",1991.0,M,RJ


In [6]:
# Novo DataFrame com Colunas a serem usadas

colunas_selecionadas = ['product_id',	'product_name',	'product_brand', 'site_category_lv1',	'review_title', 'overall_rating',	'review_text',	'reviewer_birth_year',	'reviewer_gender', 'reviewer_state']
b2w = tabela[colunas_selecionadas]

b2w.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132373 entries, 0 to 132372
Data columns (total 10 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   product_id           132373 non-null  object 
 1   product_name         132289 non-null  object 
 2   product_brand        40982 non-null   object 
 3   site_category_lv1    132367 non-null  object 
 4   review_title         132071 non-null  object 
 5   overall_rating       132373 non-null  int64  
 6   review_text          129098 non-null  object 
 7   reviewer_birth_year  126389 non-null  float64
 8   reviewer_gender      128237 non-null  object 
 9   reviewer_state       128382 non-null  object 
dtypes: float64(1), int64(1), object(8)
memory usage: 10.1+ MB


In [62]:
# Contando Caracteres das coluans a serem analisadas

desc_title = b2w['review_title'].str.len().describe()
desc_text = b2w['review_text'].str.len().describe()
print('Titulo da avaliação','\n', desc_title,'\n\n', 'Avaliação','\n', desc_text)

Titulo da avaliação 
 count    132071.000000
mean         18.782503
std          13.847694
min           1.000000
25%          10.000000
50%          16.000000
75%          23.000000
max         150.000000
Name: review_title, dtype: float64 

 Avaliação 
 count    129098.000000
mean        136.746154
std         126.364490
min          50.000000
25%          68.000000
50%          95.000000
75%         155.000000
max        4548.000000
Name: review_text, dtype: float64


In [68]:
# Textos Vazios

print('Avaliações vazias: ', b2w['review_text'].isnull().sum())

# Registros duplicados
print('Registros duplicados: ', b2w.duplicated(keep=False).sum())


# Removendo registros duplicados

b2w = b2w.drop_duplicates().reset_index(drop=True)
print('Formato do dataset após remoção de duplicados:', b2w.shape)
b2w

Avaliações vazias:  3156
Registros duplicados:  0
Formato do dataset após remoção de duplicados: (131075, 10)


,product_id,product_name,product_brand,site_category_lv1,review_title,overall_rating,review_text,reviewer_birth_year,reviewer_gender,reviewer_state
0,132532965,Notebook Asus Vivobook Max X541NA-GO472T Intel...,NaN,Informática,Bom,4,Estou contente com a compra entrega rápida o ú...,1958.0,F,RJ
1,22562178,Copo Acrílico Com Canudo 500ml Rocie,NaN,Utilidades Domésticas,"Preço imbatível, ótima qualidade",4,"Por apenas R$1994.20,eu consegui comprar esse ...",1996.0,M,SC
2,113022329,Panela de Pressão Elétrica Philips Walita Dail...,philips walita,Eletroportáteis,ATENDE TODAS AS EXPECTATIVA.,4,SUPERA EM AGILIDADE E PRATICIDADE OUTRAS PANEL...,1984.0,M,SP
3,113851581,Betoneira Columbus - Roma Brinquedos,roma jensen,Brinquedos,presente mais que desejado,4,MEU FILHO AMOU! PARECE DE VERDADE COM TANTOS D...,1985.0,F,SP
4,131788803,"Smart TV LED 43"" LG 43UJ6525 Ultra HD 4K com C...",lg,TV e Home Theater,"Sem duvidas, excelente",5,"A entrega foi no prazo, as americanas estão de...",1994.0,M,MG
...,...,...,...,...,...,...,...,...,...,...
131070,17962233,Carregador De Pilha Sony + 4 Pilhas Aa 2500mah,NaN,Câmeras e Filmadoras,Ótimo produto!,5,"Vale muito, estou usando no controle do Xbox e...",1988.0,M,RS
131071,132631701,Mop Giratório Fit + Refil Extra - At Home,NaN,Utilidades Domésticas,Sensacional,5,"Prático e barato, super indico o produto para ...",1979.0,F,SP
131072,16095859,Fita Led 5m Rgb 3528 Siliconada Com 300 Leds C...,NaN,Automotivo,Ótimo produto,4,Chegou antes do prazo previsto e corresponde a...,1979.0,F,PR
131073,6774907,Etiquetas Jurídicas Vade Mecum - Marca Fácil,marca facil,Papelaria,O produto não é bom.,1,"Material fraco, poderia ser melhor. Ficou deve...",1991.0,M,RJ
